In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
import matplotlib
import matplotlib.pyplot as plt
import time

In [2]:
# PINN 모델 정의
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(3, 16), nn.SiLU(),
            nn.Linear(16, 64), nn.SiLU(),
            nn.Linear(64, 64), nn.SiLU(),
            nn.Linear(64, 64), nn.SiLU(),
            nn.Linear(64, 16), nn.SiLU(),
            nn.Linear(16, 3)
        )

    def forward(self, x, y, t):
        input_data = torch.stack([x.view(-1), y.view(-1), t.view(-1)], dim=1)
        return self.model(input_data)

def initial_condition(x, y):
    u_initial = np.zeros_like(x)  
    v_initial = np.zeros_like(y)  
    p_initial = np.zeros_like(x)  
    return u_initial, v_initial, p_initial


# 경계 조건 (물리적 단위에 맞게 수정)
def boundary_conditions_wall_cylinder(uvp, x, y):
    u, v, p = torch.chunk(uvp, 3, dim=1)  

    u_boundary = torch.zeros_like(x) 
    v_boundary = torch.zeros_like(y)  

    # p_x = torch.autograd.grad(p.sum(), x, create_graph=True)[0]
    # p_y = torch.autograd.grad(p.sum(), y, create_graph=True)[0] 
    
    # r_length = ((x-0.2)*(x-0.2)+(y-0.2)*(y-0.2))**(1/2)
    # r_norm_x = (x-0.2)/r_length
    # r_norm_y = (y-0.2)/r_length

    # p_boundary = p_x*r_norm_x + p_y+r_norm_y
    p_boundary = torch.zeros_like(x)

    return u_boundary, v_boundary, p_boundary

# 경계 조건 (물리적 단위에 맞게 수정)
def boundary_conditions_wall_channel(uvp, x, y):
    u, v, p = torch.chunk(uvp, 3, dim=1)  

    u_boundary = torch.zeros_like(x) 
    v_boundary = torch.zeros_like(y)  

    # p_y = torch.autograd.grad(p.sum(), y, create_graph=True)[0] 
    # p_boundary = p_y
    p_boundary = torch.zeros_like(x)

    return u_boundary, v_boundary, p_boundary

def boundary_conditions_inflow(x,y):
    u_boundary = 4*0.3*(y)*(0.41-y)/(0.41*0.41) 
    #u_boundary = np.full_like(x,(2/3)*0.3)
    v_boundary = np.zeros_like(y)  
    p_boundary = np.zeros_like(x)   #standard air property --> need to edit 
    # p_boundary = np.zeros_like(x) + 0.5*1*(u_boundary)**2
    return u_boundary, v_boundary, p_boundary


def boundary_conditions_outflow(x,y):
    u_boundary = np.zeros_like(x) 
    v_boundary = np.zeros_like(y)  
    p_boundary = np.zeros_like(x)  
    return u_boundary, v_boundary, p_boundary


def navier_stokes_residual(uvp, x, y, t):
    u, v, p = torch.chunk(uvp, 3, dim=1)  

    # Constants
    rho = 1  # Density
    mu = 0.001  # Viscosity coefficient

    # Compute derivatives using automatic differentiation
    u_t = torch.autograd.grad(u.sum(), t, create_graph=True)[0]
    u_x = torch.autograd.grad(u.sum(), x, create_graph=True)[0]
    u_y = torch.autograd.grad(u.sum(), y, create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x.sum(), x, create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y.sum(), y, create_graph=True)[0]

    v_t = torch.autograd.grad(v.sum(), t, create_graph=True)[0]
    v_x = torch.autograd.grad(v.sum(), x, create_graph=True)[0]
    v_y = torch.autograd.grad(v.sum(), y, create_graph=True)[0]
    v_xx = torch.autograd.grad(v_x.sum(), x, create_graph=True)[0]
    v_yy = torch.autograd.grad(v_y.sum(), y, create_graph=True)[0]

    p_x = torch.autograd.grad(p.sum(), x, create_graph=True)[0]
    p_y = torch.autograd.grad(p.sum(), y, create_graph=True)[0] 

    # Navier-Stokes equations for u and v momentum
    residual_u = rho*(u_t + u * u_x + v * u_y) - mu*(u_xx + u_yy) + p_x
    residual_v = rho*(v_t + u * v_x + v * v_y) - mu*(v_xx + v_yy) + p_y

    # Continuity equation for incompressibility condition
    residual_p = (u_x + v_y)

    return residual_u, residual_v, residual_p

In [3]:
# 학습 데이터 생성 (물리적 단위에 맞게 수정)
x_initial = np.linspace(0, 2.2, 2200).reshape(-1, 1)
y_initial = np.linspace(0, 0.41, 410).reshape(-1, 1)

inflow_mask = x_initial>0
x_initial = x_initial[inflow_mask]

up_boundary_mask = y_initial<0.41
y_initial = y_initial[up_boundary_mask]

down_boundary_mask = y_initial>0
y_initial = y_initial[down_boundary_mask]


x_initial, y_initial = np.meshgrid(x_initial, y_initial)

#cylinder 내부면을 제외한 포인트를 생성
cylinder_mask = (x_initial-0.2) ** 2 + (y_initial-0.2) ** 2 > 0.05 ** 2
x_initial = x_initial[cylinder_mask]
y_initial = y_initial[cylinder_mask]


# cylinder 벽면 점들을 추가
cylinder_points = 1000
theta = np.linspace(0, 2 * np.pi, cylinder_points).reshape(-1, 1)
x_wall_boundary = 0.2 + np.cos(theta)*0.05  
y_wall_boundary = 0.2 + np.sin(theta)*0.05


# inflow 점들을 추가
y_inflow_boundary = np.linspace(0, 0.41, 410).reshape(-1, 1)
x_inflow_boundary = np.zeros_like(y_inflow_boundary)

# outflow 점들을 추가
y_outflow_boundary = np.linspace(0, 0.41, 410).reshape(-1, 1)
x_outflow_boundary = np.full_like(y_outflow_boundary, 2.2)

x_up_wall_boundary = np.linspace(0, 2.2, 2200).reshape(-1, 1)
inflow_mask = x_up_wall_boundary > 0
x_up_wall_boundary = x_up_wall_boundary[inflow_mask]
y_up_wall_boundary = np.full_like(x_up_wall_boundary, 0.41)


x_down_wall_boundary = np.linspace(0, 2.2, 2200).reshape(-1, 1)
inflow_mask = x_down_wall_boundary > 0
x_down_wall_boundary = x_down_wall_boundary[inflow_mask]
y_down_wall_boundary = np.full_like(x_down_wall_boundary, 0)


x_collocation = []
y_collocation = []

cnt = 0
while True:
    x_c = np.random.uniform(0, 2.2)
    y_c = np.random.uniform(0, 0.41)
    if not ((x_c-0.2) ** 2 + (y_c-0.2) ** 2 < 0.05 ** 2):
        # if (not y_c == 0):
        #     if (not y_c == 0.41):
        #         if not (x_c ==0):
        x_collocation.append(x_c)
        y_collocation.append(y_c)
        cnt += 1
        if (cnt > 50000):
            break

x_collocation = np.array(x_collocation).reshape(-1, 1)
y_collocation = np.array(y_collocation).reshape(-1, 1)

In [7]:
def train_PINN(model, x_initial, y_initial, x_wall_boundary, y_wall_boundary,x_up_wall_boundary, y_up_wall_boundary, x_down_wall_boundary, y_down_wall_boundary, x_inflow_boundary, y_inflow_boundary,x_outflow_boundary, y_outflow_boundary, x_collocation, y_collocation, epochs, learning_rate):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.MSELoss()
    min_loss = 0.5
    for epoch in range(epochs):
        optimizer.zero_grad()

        t_initial = np.zeros_like(x_initial)

        x_initial_tensor, y_initial_tensor, t_initial_tensor, \
        x_wall_boundary_tensor, y_wall_boundary_tensor, \
        x_up_wall_boundary_tensor, y_up_wall_boundary_tensor, \
        x_down_wall_boundary_tensor, y_down_wall_boundary_tensor,\
        x_inflow_boundary_tensor, y_inflow_boundary_tensor,\
        x_outflow_boundary_tensor, y_outflow_boundary_tensor = (
            torch.tensor(_, dtype=torch.float32, device="cuda", requires_grad=True) for _ in
            [x_initial, y_initial, t_initial, \
            x_wall_boundary, y_wall_boundary, \
            x_up_wall_boundary, y_up_wall_boundary, \
            x_down_wall_boundary, y_down_wall_boundary, \
            x_inflow_boundary, y_inflow_boundary,\
            x_outflow_boundary, y_outflow_boundary]
        )

        u_initial_pred = model(x_initial_tensor, y_initial_tensor, t_initial_tensor)
        u_initial_exact = torch.tensor(initial_condition(x_initial, y_initial), device="cuda", dtype=torch.float32)
        
        loss1 = loss_fn(u_initial_pred[:,0].view(-1, 1), u_initial_exact[0,:].view(-1, 1))
        loss2 = loss_fn(u_initial_pred[:,1].view(-1, 1), u_initial_exact[1,:].view(-1, 1))
        loss3 = loss_fn(u_initial_pred[:,2].view(-1, 1), u_initial_exact[2,:].view(-1, 1))

    
        loss4, loss5, loss6 = 0, 0, 0
        for t_boundary in np.linspace(0, 10, 101):
            x_wall_boundary_tensor = x_wall_boundary_tensor.squeeze()
            y_wall_boundary_tensor = y_wall_boundary_tensor.squeeze()
            
            t_boundary_tensor = torch.full_like(x_wall_boundary_tensor, t_boundary)
            
            u_wall_boundary_pred = model(x_wall_boundary_tensor, y_wall_boundary_tensor, t_boundary_tensor)
            u_wall_boundary_exact = boundary_conditions_wall_cylinder(u_wall_boundary_pred, x_wall_boundary_tensor, y_wall_boundary_tensor )


            loss4 += loss_fn(u_wall_boundary_pred[:,0].view(-1, 1), u_wall_boundary_exact[0].view(-1, 1))
            loss5 += loss_fn(u_wall_boundary_pred[:,1].view(-1, 1), u_wall_boundary_exact[1].view(-1, 1))
            loss6 += loss_fn(u_wall_boundary_exact[2].view(-1, 1), torch.zeros_like(u_wall_boundary_exact[2].view(-1, 1)))


        loss7, loss8, loss9 = 0,0,0
        for t_boundary in np.linspace(0, 10, 101):
            x_inflow_boundary_tensor = x_inflow_boundary_tensor.squeeze()
            y_inflow_boundary_tensor = y_inflow_boundary_tensor.squeeze()
            
            t_boundary_tensor = torch.full_like(x_inflow_boundary_tensor, t_boundary)

            u_inflow_boundary_pred = model(x_inflow_boundary_tensor, y_inflow_boundary_tensor, t_boundary_tensor)
            u_inflow_boundary_exact = torch.tensor(boundary_conditions_inflow(x_inflow_boundary, y_inflow_boundary ), device="cuda", dtype=torch.float32)
            
            loss7 += loss_fn(u_inflow_boundary_pred[:,0].view(-1, 1), u_inflow_boundary_exact[0,:].view(-1, 1))
            loss8 += loss_fn(u_inflow_boundary_pred[:,1].view(-1, 1), u_inflow_boundary_exact[1,:].view(-1, 1))
            #loss9 += loss_fn(u_inflow_boundary_pred[:,2].view(-1, 1), u_inflow_boundary_exact[2,:].view(-1, 1))


        loss10, loss11, loss12 = 0, 0, 0

        for t_collocation in np.linspace(0, 10, 101):
            idx = np.random.choice(len(x_collocation), size=500, replace=False)
            x_collocation_batch = x_collocation[idx]
            y_collocation_batch = y_collocation[idx]
            t_collocation_batch = np.full_like(x_collocation_batch, t_collocation)
            
            x_collocation_tensor, y_collocation_tensor, t_collocation_tensor = (
                torch.tensor(_, dtype=torch.float32, device="cuda", requires_grad=True) for _ in
                [x_collocation_batch, y_collocation_batch, t_collocation_batch]
            )
            
            x_collocation_tensor = x_collocation_tensor.squeeze()
            y_collocation_tensor = y_collocation_tensor.squeeze()
            t_collocation_tensor = t_collocation_tensor.squeeze()
            
            u_collocation_pred = model(x_collocation_tensor, y_collocation_tensor, t_collocation_tensor)
            equation_residual_u, equation_residual_v, equation_residual_p = navier_stokes_residual(u_collocation_pred, x_collocation_tensor, y_collocation_tensor, t_collocation_tensor)
            loss10 += loss_fn(equation_residual_u, torch.zeros_like(equation_residual_u))
            loss11 += loss_fn(equation_residual_v, torch.zeros_like(equation_residual_v))
            loss12 += loss_fn(equation_residual_p, torch.zeros_like(equation_residual_p))


        loss13, loss14, loss15 = 0,0,0
        loss16, loss17, loss18 = 0,0,0

        for t_boundary in np.linspace(0, 10, 101):
            x_up_wall_boundary_tensor = x_up_wall_boundary_tensor.squeeze()
            y_up_wall_boundary_tensor = y_up_wall_boundary_tensor.squeeze()
            
            t_boundary_tensor = torch.full_like(x_up_wall_boundary_tensor, t_boundary)

            u_up_wall_boundary_pred = model(x_up_wall_boundary_tensor, y_up_wall_boundary_tensor, t_boundary_tensor)
            u_up_wall_boundary_exact = boundary_conditions_wall_channel(u_up_wall_boundary_pred, x_up_wall_boundary_tensor, y_up_wall_boundary_tensor )
            
            loss13 += loss_fn(u_up_wall_boundary_pred[:,0].view(-1, 1), u_up_wall_boundary_exact[0].view(-1, 1))
            loss14 += loss_fn(u_up_wall_boundary_pred[:,1].view(-1, 1), u_up_wall_boundary_exact[1].view(-1, 1))
            loss15 += loss_fn(u_up_wall_boundary_exact[2].view(-1, 1), torch.zeros_like(u_up_wall_boundary_exact[2].view(-1, 1)))


            x_down_wall_boundary_tensor = x_down_wall_boundary_tensor.squeeze()
            y_down_wall_boundary_tensor = y_down_wall_boundary_tensor.squeeze()
            
            t_boundary_tensor = torch.full_like(x_down_wall_boundary_tensor, t_boundary)

            u_down_wall_boundary_pred = model(x_down_wall_boundary_tensor, y_down_wall_boundary_tensor, t_boundary_tensor)
            u_down_wall_boundary_exact = boundary_conditions_wall_channel(u_down_wall_boundary_pred, x_down_wall_boundary_tensor, y_down_wall_boundary_tensor )
            
            loss16 += loss_fn(u_down_wall_boundary_pred[:,0].view(-1, 1), u_down_wall_boundary_exact[0].view(-1, 1))
            loss17 += loss_fn(u_down_wall_boundary_pred[:,1].view(-1, 1), u_down_wall_boundary_exact[1].view(-1, 1))
            loss18 += loss_fn(u_down_wall_boundary_exact[2].view(-1, 1), torch.zeros_like(u_down_wall_boundary_exact[2].view(-1, 1)))


        loss19, loss20, loss21 = 0,0,0
        for t_boundary in np.linspace(0, 10, 101):
            x_outflow_boundary_tensor = x_outflow_boundary_tensor.squeeze()
            y_outflow_boundary_tensor = y_outflow_boundary_tensor.squeeze()
            
            t_boundary_tensor = torch.full_like(x_outflow_boundary_tensor, t_boundary)

            u_outflow_boundary_pred = model(x_outflow_boundary_tensor, y_outflow_boundary_tensor, t_boundary_tensor)
            u_outflow_boundary_exact = torch.tensor(boundary_conditions_outflow(x_outflow_boundary, y_outflow_boundary ), device="cuda", dtype=torch.float32)
            
            #loss19 += loss_fn(u_inflow_boundary_pred[:,0].view(-1, 1), u_inflow_boundary_exact[0,:].view(-1, 1))
            #loss20 += loss_fn(u_inflow_boundary_pred[:,1].view(-1, 1), u_inflow_boundary_exact[1,:].view(-1, 1))
            loss21 += loss_fn(u_outflow_boundary_pred[:,2].view(-1, 1), u_outflow_boundary_exact[2,:].view(-1, 1))



        loss = loss1 + loss2 + loss3 + loss4 + loss5 + loss7 + loss8 + loss10 + loss11 + loss12 + loss13 + loss14 + loss16 + loss17 + loss21

        print("initial:", loss1.item(), loss2.item(), loss3.item())
        print("wall:", loss4.item(), loss5.item())
        print("inflow:", loss7.item(), loss8.item())
        print("collocation:",loss10.item(), loss11.item(), loss12.item())
        print("up boundary:", loss13.item(), loss14.item())
        print("down boundary:", loss16.item(), loss17.item())
        print("outflow:", loss21.item())
        
        print(f"Epoch [{epoch}/{epochs}], Loss: {loss.item()}")
        print()

        if (epoch%10 == 0):
            torch.save(model.state_dict(), "model/test_current.pt")

        if (loss.item()<min_loss):
            torch.save(model.state_dict(), "model/test_current_min3.pt")
            min_loss = loss.item()
        

        loss.backward()
        optimizer.step()
        

    print("Training completed.")


In [8]:
# 모델 생성 및 학습 
model = PINN().cuda()
model.load_state_dict(torch.load("model/test_current_min2.pt"))

<All keys matched successfully>

In [9]:
train_PINN(model, x_initial, y_initial,x_wall_boundary, y_wall_boundary, x_up_wall_boundary, y_up_wall_boundary, x_down_wall_boundary, y_down_wall_boundary, x_inflow_boundary, y_inflow_boundary,x_outflow_boundary, y_outflow_boundary, x_collocation, y_collocation, epochs=5000, learning_rate=0.001)

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 10.00 GiB total capacity; 9.23 GiB already allocated; 0 bytes free; 9.29 GiB reserved in total by PyTorch)